In [13]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def binary_array_to_hex(binary_array):
    # Step 1: Convert binary array to binary string
    binary_string = ''.join(str(bit) for bit in binary_array)
    
    # Step 2: Convert binary string to integer
    integer_value = int(binary_string, 2)
    
    # Step 3: Convert integer to hexadecimal string
    hex_value = hex(integer_value)
    
    return hex_value


def get_glyph_metrics(char, font_path, font_size):
    # Load the font
    font = ImageFont.truetype(font_path, font_size)

    # Create a dummy image to facilitate drawing (not used for display)
    dummy_image = Image.new('1', (1, 1), 1)
    draw = ImageDraw.Draw(dummy_image)

    # Get size of the character
    size = draw.textsize(char, font=font)

    # Get font ascent and descent
    ascent, descent = font.getmetrics()

    # Calculate character width, height and baseline
    width, height = font.getsize(char)
    baseline = height - descent

    # Use getmask2 to get advance width and height
    mask, offset = font.getmask2(char, mode='L')
    advance_width, advance_height = mask.size

    # Return relevant metrics
    metrics = {
        "char_width": width,
        "char_height": height,
        "advance_width": advance_width,
        "advance_height": advance_height,
        "ascent": ascent,
        "descent": descent,
        "baseline": baseline,
        "offset_x": offset[0],
        "offset_y": offset[1]
    }
    return metrics
    
def render_char_to_bitmap(char, font_path, font_size):
    # Load the font
    font = ImageFont.truetype(font_path, font_size)

    # Determine size required for the character
    image_size = font.getsize(char)

    # Create a new blank image, white background
    image = Image.new('1', image_size, "white")

    # Initialize ImageDraw
    draw = ImageDraw.Draw(image)

    # Draw the character onto the image
    draw.text((0, 0), char, font=font, fill="black")

    # Convert image to numpy array
    data = np.array(image)

    # Convert non-zero (white) to 0 and zero (black) to 1
    binary_representation = 1 - data
    return binary_representation

# Usage example
bitmap = render_char_to_bitmap('y', 'arial.ttf', 12)
metrics  = get_glyph_metrics('y', 'arial.ttf', 12)
print(bitmap)
print(metrics)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 0 1 0]
 [0 1 0 1 0 0]
 [0 1 0 1 0 0]
 [0 1 0 1 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]]
{'char_width': 6, 'char_height': 13, 'advance_width': 6, 'advance_height': 9, 'ascent': 11, 'descent': 3, 'baseline': 10, 'offset_x': 0, 'offset_y': 4}


C:\Users\Musta\AppData\Local\Temp\ipykernel_36852\1221041689.py:58: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  image_size = font.getsize(char)
C:\Users\Musta\AppData\Local\Temp\ipykernel_36852\1221041689.py:26: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  size = draw.textsize(char, font=font)
C:\Users\Musta\AppData\Local\Temp\ipykernel_36852\1221041689.py:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  width, height = font.getsize(char)


In [8]:

def render_text(char, font_path, font_size, draw_pixel=0, cursor_x=0, cursor_y=0):
    font = ImageFont.truetype(font_path, font_size)

    mask, offset = font.getmask2(char, mode='L')
    
    

    advance_width, advance_height = mask.size

    # Convert the mask into a bitmap where non-zero values indicate where to draw pixels
    bytes_per_row = (advance_width + 7) // 8

    return *offset, advance_width, advance_height

render_text('y', 'arial.ttf', 12)

(0, 4, 6, 9)

In [12]:
def createCode(char, hexs, width, height, offsetX, offsetY):
    hexs_str = ", ".join(hexs)
    if width > 12:
        print("WARNING!!!!!!!")
    return f"\tarial.setGlyph({hex(char)}, {{{{{hexs_str}}}, {width}, {height}, {offsetX}, {offsetY}}}); // Glyph {hex(char)} - '{chr(char)}'"

def createGlyph(char, height=12):
    bitmap = render_char_to_bitmap(chr(char), 'arial.ttf', height)
    offsetX, offsetY, advanceX, advanceY = render_text(chr(char), 'arial.ttf', height)

    hexs = [binary_array_to_hex(x) for x in bitmap.T]
    width = len(hexs)

    return createCode(char, hexs, width, height, offsetX, offsetY)

print(createGlyph(0x79))

	arial.setGlyph(0x79, {{0x180, 0x71, 0xe, 0x70, 0x180, 0x0}, 6, 12, 0, 4}); // Glyph 0x79 - 'y'


C:\Users\Musta\AppData\Local\Temp\ipykernel_36852\2899712230.py:58: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  image_size = font.getsize(char)


In [10]:
for i in range(0x21, 0x7F):
    print(createGlyph(i))

C:\Users\Musta\AppData\Local\Temp\ipykernel_36852\2899712230.py:58: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  image_size = font.getsize(char)


	arial.setGlyph(0x21, {{0x0, 0x1fd, 0x0}, 3, 12, 0, 2}); // Glyph 0x21 - !
	arial.setGlyph(0x22, {{0x1c0, 0x0, 0x1c0, 0x0}, 4, 12, 0, 2}); // Glyph 0x22 - "
	arial.setGlyph(0x23, {{0x48, 0x4b, 0x7c, 0x1cb, 0x7c, 0x1c8, 0x48}, 7, 12, 0, 2}); // Glyph 0x23 - #
	arial.setGlyph(0x24, {{0x0, 0x1cc, 0x222, 0x3ff, 0x222, 0x11c, 0x0}, 7, 12, 0, 2}); // Glyph 0x24 - $
	arial.setGlyph(0x25, {{0x0, 0xe0, 0x110, 0x111, 0xe6, 0x38, 0xce, 0x111, 0x11, 0xe, 0x0}, 11, 12, 0, 2}); // Glyph 0x25 - %
	arial.setGlyph(0x26, {{0x0, 0xe, 0xf1, 0x111, 0x129, 0xc6, 0xa, 0x1}, 8, 12, 0, 2}); // Glyph 0x26 - &
	arial.setGlyph(0x27, {{0x0, 0x1c0}, 2, 12, 0, 2}); // Glyph 0x27 - '
	arial.setGlyph(0x28, {{0x0, 0xf8, 0x306, 0x401}, 4, 12, 0, 2}); // Glyph 0x28 - (
	arial.setGlyph(0x29, {{0x401, 0x306, 0xf8, 0x0}, 4, 12, 0, 2}); // Glyph 0x29 - )
	arial.setGlyph(0x2a, {{0x80, 0xa0, 0x1c0, 0xa0, 0x80}, 5, 12, 0, 2}); // Glyph 0x2a - *
	arial.setGlyph(0x2b, {{0x0, 0x10, 0x10, 0x7c, 0x10, 0x10, 0x0}, 7, 12, 0, 4}); // G